In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from pathlib import Path

import lightning.pytorch as pl
import torch
import wandb
from sdofm import utils
from sdofm.datasets import SDOMLDataModule, DegradedSDOMLDataModule
from sdofm.pretraining import MAE, SAMAE
from sdofm.finetuning import Autocalibration

/home/walsh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import omegaconf

cfg = omegaconf.OmegaConf.load("../experiments/pretrain_32.2M_samae_tpu.yaml")

In [4]:
# from sdofm.utils import flatten_dict
# import yaml

# data = flatten_dict(cfg, sep="___")
# with open('testingout.yaml', 'w+') as outfile:
#     yaml.dump(data, outfile, default_flow_style=False)
# outfile.close()

In [5]:
# from omegaconf import OmegaConf
# OmegaConf.save(config, "testout.yaml")

In [4]:
degraded_data_module = DegradedSDOMLDataModule(
    hmi_path=None,
    aia_path=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.aia
    ),
    eve_path=None,
    components=cfg.data.sdoml.components,
    wavelengths=cfg.data.sdoml.wavelengths,
    ions=cfg.data.sdoml.ions,
    frequency=cfg.data.sdoml.frequency,
    batch_size=cfg.model.opt.batch_size,
    num_workers=cfg.data.num_workers,
    val_months=cfg.data.month_splits.val,
    test_months=cfg.data.month_splits.test,
    holdout_months=cfg.data.month_splits.holdout,
    cache_dir=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.cache
    ),
    min_date=cfg.data.min_date,
    max_date=cfg.data.max_date,
    num_frames=1,
)
degraded_data_module.setup()

[* CACHE SYSTEM *] Found cached index data in /mnt/sdoml/cache/aligndata_AIA_FULL_12min.csv.
[* CACHE SYSTEM *] Found cached normalization data in /mnt/sdoml/cache/normalizations_AIA_FULL_12min.json.
[* CACHE SYSTEM *] Found cached HMI mask data in /mnt/sdoml/cache/hmi_mask_512x512.npy.


In [5]:
import yaml
from pathlib import Path

conf = yaml.safe_load(
    Path(
        "/home/walsh/repos/SDO-FM/output/2024-04-17-00-41-29/0/wandb/latest-run/files/config.yaml"
    ).read_text()
)
backbone_cfg = utils.unflatten_dict(conf, sep="___", wandb_mode=True)
backbone_cfg

FileNotFoundError: [Errno 2] No such file or directory: '/home/walsh/repos/SDO-FM/output/2024-04-17-00-41-29/0/wandb/latest-run/files/config.yaml'

In [8]:
backbone_cfg.model.opt

{'loss': 'mse',
 'scheduler': 'constant',
 'scheduler_warmup': 0,
 'batch_size': 3,
 'learning_rate': 0.0001,
 'weight_decay': 0.0003,
 'optimiser': 'adam',
 'epochs': 4,
 'patience': 2}

In [6]:
backbone_cfg=cfg
backbone = SAMAE.load_from_checkpoint(
    **backbone_cfg.model.mae,
    **backbone_cfg.model.samae,
    optimiser=backbone_cfg.model.opt.optimiser,
    lr=backbone_cfg.model.opt.learning_rate,
    weight_decay=backbone_cfg.model.opt.weight_decay,
    checkpoint_path="/home/walsh/SDO-FM/outputs/2024-05-23/22-51-44/lightning_logs/version_0/checkpoints/epoch=2-step=132.ckpt"
)

NotImplementedError: Could not run 'aten::empty.memory_format' with arguments from the 'XLA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::empty.memory_format' is only available for these backends: [CPU, CUDA, Meta, QuantizedCPU, QuantizedCUDA, QuantizedMeta, MkldnnCPU, SparseCPU, SparseCUDA, SparseMeta, SparseCsrCPU, SparseCsrCUDA, SparseCsrMeta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, Aut

In [16]:
model = Autocalibration(
    **cfg.model.mae,
    **cfg.model.dimming,
    optimiser=cfg.model.opt.optimiser,
    lr=cfg.model.opt.learning_rate,
    weight_decay=cfg.model.opt.weight_decay,
    backbone=backbone,
    hyperparam_ignore=['backbone']
)

In [17]:
trainer = pl.Trainer(
    devices=1, accelerator=cfg.experiment.accelerator, max_epochs=cfg.model.opt.epochs
)
trainer.fit(model=model, datamodule=degraded_data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name          | Type                                 | Params
-----------------------------------------------------------------------
0 | backbone      | SAMAE                                | 32.2 M
1 | encoder       | PrithviEncoder                       | 32.2 M
2 | decoder       | ConvTransformerTokensToEmbeddingNeck | 28.9 K
3 | head          | Autocalibration13                    | 93.4 K
4 | loss_function | MSELoss                              | 0     
-----------------------------------------------------------------------
122 K     Trainable params
32.2 M    Non-trainable params
32.3 M    Total params
129.203   Total estimated model params size (MB)


ValueError: dictionary update sequence element #0 has length 1; 2 is required